In [1]:
from dotenv import load_dotenv
import os
# os.chdir("/mnt/e/Projects/Vinal-hackathon")
# Load environment variables from .env file
load_dotenv()


True

In [2]:

# # Retrieve Azure OpenAI credentials
# azure_openai_key = os.getenv("AZURE_OPENAI_KEY")
# azure_openai_endpoint = os.getenv("AZURE_OPENAI_ENDPOINT")
# azure_openai_api_version = os.getenv("AZURE_OPENAI_API_VERSION")

# # Ensure the variables are loaded
# if not all([azure_openai_key, azure_openai_endpoint, azure_openai_api_version]):
#     raise ValueError("Missing one or more Azure OpenAI environment variables.")

In [3]:
from langchain_openai import AzureChatOpenAI

llm = AzureChatOpenAI(
    azure_deployment="gpt-4o",  # or your deployment
    api_version="2023-12-01-preview",  # or your api version
    temperature=0,
    max_tokens=None,
    timeout=None,
    max_retries=4,
    # other params...
)

In [4]:
llm.invoke("What is the capital of France?").content

'The capital of France is **Paris**.'

In [5]:
from PIL import Image
import json
from langchain.schema import HumanMessage, SystemMessage
import base64

# Define the file path
file_path = "user_image.png"


In [24]:
"""
event_poster_analyser.py
────────────────────────
• Sends an image to GPT‑4‑o (or any vision‑capable model)
• Expects a strict JSON reply defined by StructuredOutputParser
• Auto‑fixes malformed JSON (OutputFixingParser)
• Auto‑retries the whole call up to N times (RetryOutputParser)
No optional imports – runs on old or new LangChain releases.
"""

import base64, json, traceback
from pathlib import Path

from langchain.chat_models import ChatOpenAI
from langchain.schema import SystemMessage, HumanMessage
from langchain.prompts import ChatPromptTemplate   # ← always available

from langchain.output_parsers import (
    ResponseSchema,
    StructuredOutputParser,
    OutputFixingParser,
    RetryOutputParser,
)

# ───────────────────────── vision LLM ─────────────────────────
llm = llm

# ───────────────────── JSON schema & parsers ──────────────────
schemas = [
    ResponseSchema(name="is_valid_event_poster",
                   description="true if the image shows a legitimate event poster, false otherwise"),
    ResponseSchema(name="justification",
                   description="One‑sentence rationale"),
    ResponseSchema(name="event_name",
                   description="Event name or null"),
    ResponseSchema(name="event_datetime_iso",
                   description="Event date‑time ISO‑8601 or null"),
]

base_parser  = StructuredOutputParser.from_response_schemas(schemas)
fix_parser   = OutputFixingParser.from_llm(parser=base_parser,  llm=llm)
retry_parser = RetryOutputParser.from_llm(  parser=fix_parser, llm=llm, max_retries=2)
FORMAT = base_parser.get_format_instructions()

# ─────────────────── helper: file → data‑URL ─────────────────
def to_data_url(img_path: str | Path) -> str:
    img_path = Path(img_path)
    mime = f"image/{img_path.suffix.lstrip('.') or 'png'}"
    data = base64.b64encode(img_path.read_bytes()).decode()
    return f"data:{mime};base64,{data}"

# ─────────────────── main analyse routine ────────────────────
def analyse_event_poster(image_path: str | Path) -> dict:
    try:
        # 1. craft the chat messages
        data_url = to_data_url(image_path)
        system_msg = SystemMessage(content="You are an AI assistant that analyses event posters., todays date is 20 April 2025")
        human_msg  = HumanMessage(
            content=[
                {"type": "text",
                 "text": f"{FORMAT}\n\nAnalyse the image below and respond:"},
                {"type": "image_url", "image_url": {"url": data_url}},
            ]
        )
        messages = [system_msg, human_msg]

        # 2. turn messages → PromptValue WITHOUT importing ChatPromptValue
        prompt_value = ChatPromptTemplate.from_messages(messages).format_prompt()

        # 3. call the model
        raw_output = llm(messages).content

        # 4. parse / auto‑fix / auto‑retry
        return retry_parser.parse_with_prompt(raw_output, prompt_value)

    except Exception as exc:
        return {
            "error": f"{type(exc).__name__}: {exc}",
            "traceback": traceback.format_exc(),
        }


In [25]:

# ───────────────────── quick CLI test ───────────────────────
if __name__ == "__main__":
    test_path = "/mnt/e/Projects/Vinal-hackathon/user_image_2.png"   # change as needed
    result = analyse_event_poster(test_path)
    print(json.dumps(result, indent=2, ensure_ascii=False))


{
  "is_valid_event_poster": "true",
  "justification": "The image contains clear details about an event, including the name, date, time, and location.",
  "event_name": "JSOM Spring Fling",
  "event_datetime_iso": "2025-04-17T15:30:00"
}


In [20]:
def create_event_ics(result: dict,
                     filename: str = "event.ics",
                     default_duration_minutes: int = 60) -> bool:
    from datetime import datetime, timedelta, timezone
    from pathlib import Path

    if str(result.get("is_valid_event_poster", "")).lower() != "true":
        return False

    start_dt = datetime.fromisoformat(result["event_datetime_iso"])
    if start_dt.tzinfo is None:
        start_dt = start_dt.replace(tzinfo=timezone.utc)
    end_dt = start_dt + timedelta(minutes=default_duration_minutes)

    # formatter
    zfmt = "%Y%m%dT%H%M%SZ"
    def z(dt): return dt.astimezone(timezone.utc).strftime(zfmt)

    ical_lines = [
        "BEGIN:VCALENDAR",
        "VERSION:2.0",
        "PRODID:-//EventPosterParser 1.0//EN",
        "METHOD:PUBLISH",                        # <-- NEW
        "CALSCALE:GREGORIAN",
        "BEGIN:VEVENT",
        f"UID:{int(start_dt.timestamp())}@eventposter",
        f"DTSTAMP:{z(datetime.utcnow())}",
        f"DTSTART:{z(start_dt)}",
        f"DTEND:{z(end_dt)}",
        f"SUMMARY:{result.get('event_name','Untitled Event')}",
        "END:VEVENT",
        "END:VCALENDAR"
    ]

    # Write with CRLF endings
    Path(filename).write_text("\r\n".join(ical_lines) + "\r\n",
                              encoding="utf-8")
    return True


In [21]:
success = create_event_ics(result)   # writes event.ics, success == True


/tmp/ipykernel_21387/4191746347.py:27: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  f"DTSTAMP:{z(datetime.utcnow())}",


In [8]:
import smtplib, os
from email.mime.multipart import MIMEMultipart
from email.mime.text import MIMEText
from email.mime.application import MIMEApplication   # good enough for .ics

# ------------------------------------------------------------------
# 1) CONFIGURE THESE FOUR LINES ONLY --------------------------------
SENDER   = "veganman2018@gmail.com"
APP_PWD  = "ckwvrboupappkpdc"               # 16‑chars, no spaces
RECIPIENTS = [
    "Vinal.Jain@UTDallas.edu",
    "vxj230003@UTDallas.edu",
    "virat-kumar@outlook.com",
    "vk001716@gmail.com"
]
ICS_PATH = "/mnt/e/Projects/Vinal-hackathon/event.ics"
# ------------------------------------------------------------------

# -- Build the message ------------------------------------------------
msg               = MIMEMultipart("mixed")
msg["From"]       = SENDER
msg["To"]         = ", ".join(RECIPIENTS)
msg["Subject"]    = "Add your event"

# 1️⃣  Body (plain text helps spam filters)
body_text = (
    "Hi team,\n\n"
    "Kindly find the calendar invite attached. "
    "Open it to add the event to your calendar.\n\n"
    "Regards,\nVirat\n"
)
msg.attach(MIMEText(body_text, "plain"))

# 2️⃣  Attachment (.ics)
basename = os.path.basename(ICS_PATH)
with open(ICS_PATH, "rb") as f:
    part = MIMEApplication(f.read(), Name=basename)
    part['Content-Disposition'] = f'attachment; filename="{basename}"'
    part['Content-Type']       = 'text/calendar; method=REQUEST; name="{}"'.format(basename)
    msg.attach(part)

# -- Send -------------------------------------------------------------
with smtplib.SMTP_SSL("smtp.gmail.com", 465) as smtp:
    smtp.login(SENDER, APP_PWD)
    result = smtp.sendmail(SENDER, RECIPIENTS, msg.as_string())

print("Gmail response dict (empty means accepted):", result)


Gmail response dict (empty means accepted): {}
